In [ ]:
import sys
import os

project_root = 'D:\working\MathAgent'
if project_root not in sys.path:
    sys.path.insert(0, project_root)


## PdfRarserNode测试

In [ ]:
num = "06"
##%%
shared = {
    "input_pdf_folder_path": "../../data/rag_data/reference1",
    "txt_path": f"../../output/RAG/test_output{num}.txt",
    "json_path": f"../../output/RAG/test_output{num}.json",
    "md_path": f"../../output/RAG/test_output{num}.md"

}
shared

In [ ]:
from code.RAG.node import PdfParseNode

read_node = PdfParseNode()


In [ ]:

read_node.run(shared)

## rewrite测试


In [ ]:
from code.RAG.node import RewriteNode

rewrite_node = RewriteNode()
##%%
rewrite_node.run(shared)

In [ ]:
num = 1
##%%
shared = {
    "input_pdf_folder_path": "../../data/rag_data/references",
    "txt_path": f"../../output/Bluebook/test_output{num}.txt",
    "json_path": f"../../output/Bluebook/test_output{num}.json",
    "md_path": f"../../output/Bluebook/test_output{num}.md"
}
shared

### 纠错：reference里有重复文件

In [ ]:
import json

# 定义 JSON 文件路径
json_path = "../../output/Bluebook/test_output1.json"

# 1. 读取 JSON 文件
with open(json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 检查 data 是否为字典
if isinstance(data, dict):
    # 2. 删除指定的 key
    key_to_delete = "高中卷5不等式的解题方法与技巧(1)"
    if key_to_delete in data:
        del data[key_to_delete]
        print(f"成功删除键: {key_to_delete}")
    else:
        print(f"键不存在: {key_to_delete}")
else:
    raise TypeError("JSON 文件内容不是一个字典")

# 3. （可选）将修改后的数据写回文件（备份原文件更安全）
with open(json_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("JSON 文件已更新。")

In [ ]:
import json
import re

# 删除新浪微博@郑剑雄...首部
HEADER_PATTERN = r'^.*?新浪微博@郑剑雄\s*\n\n'


def remove_weibo_header(text):
    return re.sub(HEADER_PATTERN, '', text, count=1, flags=re.DOTALL)


# 删除所有 “含‘群’和数字的整行”
ADLINE_PATTERN = r'^.*群.*\d+.*\n'


def remove_group_lines(text):
    # MULTILINE保证每行的^和$匹配行首尾
    return re.sub(ADLINE_PATTERN, '', text, flags=re.MULTILINE)


def clean_json_ads(input_json_path, output_json_path):
    with open(input_json_path, encoding='utf-8') as f:
        data = json.load(f)
    for key, value in data.items():
        if isinstance(value, list):
            new_list = []
            for s in value:
                if isinstance(s, str):
                    s = remove_weibo_header(s)
                    s = remove_group_lines(s)
                    # 可进一步去除多余空行
                    s = re.sub(r'\n+', '\n', s).strip()
                new_list.append(s)
            data[key] = new_list
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


if __name__ == '__main__':
    clean_json_ads("../../output/Bluebook/test_output1.json", "../../output/Bluebook/test_output2.json")

## ChunkNode测试

In [ ]:
test_num = 2
##%%
shared = {
    "txt_path": "../../output/十年真题 常用定理/test_output06.txt",
    "chunks_path": f"../../output/knowledgebase/chunks{test_num}.json"
}
shared
##%%
from code.RAG.node import RecursiveChunkNode

chunk_node = RecursiveChunkNode()

chunk_node.run(shared)

## EmbedNode测试

In [ ]:
shared["dense_db_path"] = "../../output/knowledgebase/embedding1.index"
shared
##%%
from code.RAG.node import EmbedNode

embed_node = EmbedNode()
embed_node.run(shared)

## DenseRetrieveNode测试

In [ ]:
shared["question"] = """
**已知函数 \( f(x) = x^4 - 4x^2 + 2 \)。**

1. 求 \( f(x) \) 的最小值及取得最小值的 \( x \)；
2. 设 \( a, b, c, d \) 是该函数的所有实数极值点和零点，求 \( S = a^2 + b^2 + c^2 + d^2 \) 的值。"""

shared
##%%
from code.RAG.node import DenseRetrieveNode

dense_retrieve_node = DenseRetrieveNode()
dense_retrieve_node.run(shared)

shared

# len(shared["top_k_docs"])

In [ ]:
from node import RerankNode

##%%
# 填充更像真实文档的内容
shared = {
    "questions": [
        "什么是机器学习？",
        "如何煮鸡蛋？"
    ],
    "top_k_docs_list": [
        [  # 问题1的topk文档
            "机器学习是人工智能的一个分支，致力于使计算机从数据中学习模式，并据此做出预测或决策。",
            "常见的机器学习算法包括决策树、支持向量机、和神经网络等。",
            "深度学习是机器学习的一个子领域，近年来取得了巨大成果。"
        ],
        [  # 问题2的topk文档
            "煮鸡蛋时，先将鸡蛋放入冷水锅中，水沸腾后继续煮5~7分钟。",
            "为了让鸡蛋更容易剥壳，可以在煮好后立即将鸡蛋放入冷水中冷却。",
            "鸡蛋的新鲜度会影响煮蛋效果，建议使用新鲜鸡蛋。",
            "喜欢溏心蛋的可以适当缩短煮制时间。"
        ]
    ]
}

node = RerankNode()
res = node.run(shared)

from pprint import pprint

print("返回值：", res)
print("shared['top_n_docs_list']：")
pprint(shared["top_n_docs_list"])

# 测试

In [1]:
from main import *

In [2]:
shared = make_shared(
    input_pdf_folder_path="../../data/rag_data/reference2",
    json_path="../../output/十年真题 常用定理/test_output06.json",
    txt_path="../../output/十年真题 常用定理/test_output08.txt",
    md_path="../../output/十年真题 常用定理//test_output08.md",
    chunks_path="../../output/knowledgebase/chunks3.json",
    dense_db_path="../../output/knowledgebase/embedding_db2.index",
    bm25_db_path="../../output/knowledgebase/bm25_db2.json",
    cluster_db_path="",
    question=""
)

In [17]:
# 索引模式
mode = Mode.hybrid

## AIME测试

In [4]:
# 读取数据集
json_input_path_1 = "../../data/AIME/AIME_1983_2025_10.json"
json_input_path_2 = "../../data/AIME/AIME_1983_2025_100.json"
json_input_path_3 = "../../data/AIME/AIME_1983_2025.json"
# 读取原文件到 list
with open(json_input_path_1, 'r', encoding='utf-8') as f1:
    data_10 = json.load(f1)
with open(json_input_path_2, 'r', encoding='utf-8') as f2:
    data_100 = json.load(f2)
with open(json_input_path_3, 'r', encoding='utf-8') as f3:
    data_all = json.load(f3)

question_10 = []
question_100 = []
question_all = []
for item in data_10:
    question_10.append(item['question'])
for item in data_100:
    question_100.append(item['question'])
for item in data_all:
    question_all.append(item['question'])

In [5]:
# 测试单个RAG运行
res1 = handle_question(shared, question_10[0], mode)
print(res1)

{'id': None, 'context': '= f(2\\pi) = 2$，且 $1 < \\pi - 2 < 8 - 2\\pi < 2$，  \n所以原不等式成立当且仅当：\n$$\nx \\in [\\pi - 2,\\ 8 - 2\\pi]\n$$\n\n---### 6. 复数 $z$ 满足 $|z| = 1$，使得关于 $x$ 的方程 $zx^2 + 2zx + 2 = 0$ 有实根，则这样的复数 $z$ 的和为 $\\boxed{-\\dfrac{3}{2}}$。\n解得：\n$$\n\\text{Re}(z) = 1,\\quad \\text{Im}(z) = 2.\n$$\n所以：\n$$\n|z| = \\sqrt{1^2 + 2^2} = \\sqrt{5}.\n$$\n\n---### 3. 函数性质问题\n设函数 $f(x)$ 定义在 $\\mathbb{R}$ 上，若 $f(x) + x^2$ 是奇函数，$f(x) + 2x$ 是偶函数，求 $f(1)$ 的值。\n\n**答案：** $-\\dfrac{7}{4}$\n\n**解析：**\n由条件得方程组：\n$$\n\\begin{cases}\nf(1) + 1 = -[f(-1) + 1], \\\\\nf(1) + 2 = f(-1) + 2.\n\\end{cases}\n$$\n解得：\n$$\nf(1) = -\\frac{7}{4},\\quad f(-1) = -\\frac{1}{4}.\n$$\n\n---\n\n### 4. 三角形问题\n在 $\\triangle ABC$ 中，$\\sin A = 2\\sin C$，且边长 $a,b,c$ 成等比数列，求 $\\cos A$ 的值。\n\n**答案：** $-\\dfrac{\\sqrt{2}}{4}$\n为：\n$$\n\\{\\alpha \\in \\mathbb{C} : |\\alpha| > 2\\}\n$$\n\n\n# 数学竞赛解析文档（Markdown + LaTeX 格式）\n\n## 一、复数函数的单射性分析设存在复数 $z_1, z_2$，满足 $|z_1|, |z_2| < 1$，且 $z_1 \\neq z_2$，使得 $f_\\alpha(z_1) = f_\\alpha

In [6]:
res1.get("docs")

In [7]:
# 选择data
# test_data = data_10
test_data = data_100
# test_data=data_all

In [21]:
# 测试批量RAG运行
res2 = batch_run(shared, test_data, mode)

答题进度: 100%|██████████| 100/100 [11:19<00:00,  6.79s/it]


所有问题处理完毕，用时 679.04 秒。


In [14]:
res2[0].keys()

dict_keys(['id', 'context', 'solution', 'answer'])

In [22]:
results = []
score = 0
id1standard = {d['id']: d['answer'] for d in test_data}

for res in res2:
    id_ = res['id']
    our_answer = res.get("answer")  #or res.get("rag_answer")  # 兼容你可能的不同字段
    context = res.get("context")
    solution = res.get("solution")
    standard_answer = id1standard.get(id_)
    if standard_answer is None:
        print(f"warning: id {id_} 找不到标准答案，略过")
        continue
    is_correct = AIME_evaluate(standard_answer, our_answer)
    if is_correct:
        score += 1
    results.append({
        "id": id_,
        "correct_answer": standard_answer,
        "answer": our_answer,
        "is_answer_correct": is_correct,
        "context": context,
        "solution": solution
    })
acc = score / len(results)
print(f"score={score}")
print(f"acc={acc:.4f}")


score=38
acc=0.3800


In [23]:
# 将结果保存为json文件
num = "3_100"
json_output_path = f"../../output/rag_test/AIME_evaluation_results{num}.json"

import os

with open(json_output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

## 测试Base

In [ ]:
base_model = "qwen-max-latest"

In [ ]:
from utils.llm import call_llm_stream
from utils.prompt_templates import RAG_BASEMODEL_PROMPT

